In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from utils import *
    
import os

from typing import List, Dict

In [2]:
from nlpcda.tools.Basetool import Basetool
from nlpcda.config import similarword_path
import nlpcda


class AddSimilarWord(Basetool):
    '''
    句中随意添加近义词，生成【语义重复】类型病句。
    '''

    def __init__(self, base_file=similarword_path, create_num=5, change_rate=0.05, seed=1):
        super(AddSimilarWord, self).__init__(base_file, create_num, change_rate, seed)


    def load_paser_base_file(self):
        combine_dict = {}
        for line in open(self.base_file, "r", encoding='utf-8'):
            seperate_word = line.strip().split(" ")
            # 仅保留真正近义词，过滤相关词和独立词
            if not seperate_word[0].endswith("="):
                continue
            num = len(seperate_word)
            for i in range(1, num):
                wi = seperate_word[i]
                # add to user dict
                if len(wi) > 1: self.add_word(wi)
                combine_dict[wi] = seperate_word[1:]
        print('load :%s done' % (self.base_file))
        return combine_dict

    def replace(self, replace_str:str):
        replace_str = replace_str.replace('\n', '').strip()
        seg_list = self.jieba.cut(replace_str, cut_all=False)
        words = list(seg_list)
        sentences = [replace_str]
        t = 0
        while len(sentences) < self.create_num:
            t += 1
            a_sentence = ''
            for word in words:
                a_sentence += self.s1(word)
            if a_sentence not in sentences:
                sentences.append(a_sentence)
            if t > self.create_num * self.loop_t / self.change_rate:
                break
        return sentences

    def s1(self, word:str):
        # 替换所有在combine_dict中的
        if len(word) == 1: return word
        if word in self.base_file_mapobj and self.random.random() < self.change_rate:
            wi = self.random.randint(0, len(self.base_file_mapobj[word]) - 1)
            place = self.base_file_mapobj[word][wi]
            if place == word:
                return word
            return word + place if self.random.random() < 0.5 else place + word
        else:
            return word

Simbert不能正常使用，除非你安装：bert4keras、tensorflow ，为了安装快捷，没有默认安装.... No module named 'bert4keras'


In [3]:
sw = nlpcda.Similarword()
sw.replace('努力学习文化知识能使人们的妄想在未来变成现实。')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\holaj\AppData\Local\Temp\jieba.cache
Loading model cost 0.719 seconds.
Prefix dict has been built successfully.


load :d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\nlpcda\data\同义词.txt done


['努力学习文化知识能使人们的妄想在未来变成现实。',
 '努力学习文化知识能使人人的妄想在未来改为现实。',
 '努力学习文化知识能使人们的妄想在奔头儿变成现实。',
 '努力学习文化知识能使人们的痴想在未来变成现实。',
 '努力学习知知识能使人们的妄想在未来变成现实。']

In [12]:
from base64 import encode
from shortuuid import uuid


logic_pairs = [
    ['不仅', '尽管', '虽然', '因为', '无论', '只要'], 
    ['不但', '尽管', '虽然', '因为', '无论', '只有'], 
    ['而且', '还是', '所以', '却', '就', '因此', '才'], 
    ['而且', '仍然', '所以', '却', '就', '因此', '才'], 
    ['而且', '但是', '所以', '却', '就', '因此', '才'], 
    ['而且', '可是', '所以', '却', '就', '因此', '才'], 
]


nlpcda_package_path = 'D:/Apps/Anaconda3/envs/general-torch/Lib/site-packages/nlpcda'
file_path = os.path.join(nlpcda_package_path, 'data')
file_name = 'logic_pairs.txt'
logic_pairs_file = os.path.join(file_path, file_name)

with open(logic_pairs_file, 'w', encoding='utf-8') as f:
    for line in logic_pairs:
        line = ' '.join(line)
        line = uuid()[:8] + '= ' + line + '\n'
        f.write(line)

In [21]:
text = '只有离开那个控制你人生的人，你就能重获新生。'

logic_pairs_swapper = nlpcda.Similarword(base_file=logic_pairs_file, change_rate=0.1)
logic_pairs_swapper.replace(text)[-1]

load :D:/Apps/Anaconda3/envs/general-torch/Lib/site-packages/nlpcda\data\logic_pairs.txt done


'尽管离开那个控制你人生的人，你就能重获新生。'